# start with graphframe 

$ find venv/ -name *graphx*.jar

  venv//spark-2.4.4-bin-without-hadoop/jars/spark-graphx_2.11-2.4.4.jar

$ ./dev-scripts/okta_spark jupyter-notebook --packages graphframes:graphframes:0.8.0-spark2.4-s_2.11



## data sources:
* scan 
* coverage-anommaly
* sticky-clients



In [1]:
import pyspark.sql.functions as F
from pyspark.sql.functions import udf, size, avg, count, col,sum, explode
import json

env = "production"
#s3://mist-aggregated-stats-production/aggregated-stats/top_1_time_epoch_by_site_ap_ap2_band/dt=2020-03-14/hr=16/part-00000-76298f84-5414-49c5-88f6-a981ff06d94d-c000.csv .

s3_bucket = "s3://mist-aggregated-stats-{env}/aggregated-stats/".format(env=env)
date_day = "2020-08-21"
hr = '*'

ap_neighbors_path = "top_1_time_epoch_by_site_ap_ap2_band/dt={day}/hr={hr}/*.csv".format(env=env, day=date_day, hr=hr)

ap_neighbors_path = s3_bucket + ap_neighbors_path
print(ap_neighbors_path)



# date_day = "2020-08-11"
# hr = '*'

s3_coverage_bucket = "s3://mist-secorapp-{env}/sle-coverage-anomaly/sle-coverage-anomaly-{env}/".format(env=env)
s3_coverage_path = s3_coverage_bucket + "dt={day}/hr={hr}/*.seq".format(day=date_day, hr=hr)
print(s3_coverage_path)


s3_sticky_bucket = "s3://mist-secorapp-{env}/sticky-client/sticky-client-{env}/".format(env=env)
s3_sticky_path = s3_sticky_bucket + "dt={day}/hr={hr}/*.seq".format(day=date_day, hr=hr)
print(s3_sticky_path)


# user_org_id = "38a18d4d-1623-4985-86d0-1bb06e5e2a48"  # UPS
# user_site_USNYELE = '9aa6ecdb-ddee-41b4-a8d9-872d7962b3c1'  #UPS USNYELE

# # user_org_id = '38a18d4d-1623-4985-86d0-1bb06e5e2a48'  # UPS
# user_site_USFLLON = 'c23667f3-47e9-44aa-9761-de49d34ed9f9' # USFLLON


s3://mist-aggregated-stats-production/aggregated-stats/top_1_time_epoch_by_site_ap_ap2_band/dt=2020-08-21/hr=*/*.csv
s3://mist-secorapp-production/sle-coverage-anomaly/sle-coverage-anomaly-production/dt=2020-08-21/hr=*/*.seq
s3://mist-secorapp-production/sticky-client/sticky-client-production/dt=2020-08-21/hr=*/*.seq


#  coverage-anomaly

In [5]:
# df_coverage_t = sqlContext.read.format('sequencefile').load(s3_coverage_path)
# df_coverage_t.printSchema()

In [2]:
# Coverage-anomaly 
rdd_coverage = spark.sparkContext.sequenceFile(s3_coverage_path)
# rdd_coverage.first()

df_coverage = rdd_coverage.map(lambda x: json.loads(x[1])).toDF() #.map(lambda x: json.loads(x[1])).

df_coverage_0 = df_coverage.filter(col("band")=="5")\
    .select("org", "site", "ap", "anomaly_type", "band", "error_rate", 
            "rssi_mean", "rssi_mean_base", "rssi_deviation", "sle_coverage", "sle_coverage_base", "sle_coverage_anomaly_score")


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.sequenceFile.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 1 times, most recent failure: Lost task 0.0 in stage 0.0 (TID 0, localhost, executor driver): java.io.IOException: Failed to connect to ip-192-168-1-57.ec2.internal:62800
	at org.apache.spark.network.client.TransportClientFactory.createClient(TransportClientFactory.java:245)
	at org.apache.spark.network.client.TransportClientFactory.createClient(TransportClientFactory.java:187)
	at org.apache.spark.rpc.netty.NettyRpcEnv.org$apache$spark$rpc$netty$NettyRpcEnv$$downloadClient(NettyRpcEnv.scala:368)
	at org.apache.spark.rpc.netty.NettyRpcEnv$$anonfun$openChannel$1.apply$mcV$sp(NettyRpcEnv.scala:336)
	at org.apache.spark.rpc.netty.NettyRpcEnv$$anonfun$openChannel$1.apply(NettyRpcEnv.scala:335)
	at org.apache.spark.rpc.netty.NettyRpcEnv$$anonfun$openChannel$1.apply(NettyRpcEnv.scala:335)
	at org.a

In [ ]:
df_coverage_0.show(5)

In [8]:

# df_coverage = spark.read.format("sequenceFile")\
#    .option("header", "true")\
#    .option("inferSchema", "true")\
#    .load(s3_coverage_path)
# df_coverage.createOrReplaceTempView("sequenceFile")
# df_Schema = df_scan.schema



# rrm-graph

In [ ]:
# rdd_ap_neigbhors = spark.sparkContext.textFile(ap_neighbors_path)
# rdd_ap_neigbhors.take(3)

In [ ]:
# rdd_rrm_local.map(lambda line: (line.split(',')[0], line.split(',')[1])).collect()

In [ ]:

df_scan = spark.read.format("csv")\
   .option("header", "true")\
   .option("inferSchema", "true")\
   .load(ap_neighbors_path)
df_scan.createOrReplaceTempView("scan_data")
df_Schema = df_scan.schema

In [ ]:
df_Schema

In [ ]:
df_scan.show()

In [ ]:
df_scan.describe().show()

In [ ]:
df_scan.count()

# test-site

In [ ]:
site_id = "5e8fe474-a9ee-4d01-a2b6-b022b0f9c869"  # GEG1 , AmazonOTFC-prod

site_id = "a7092875-257f-43f3-9514-ca1ab688bec0" # Sam's club. 4989

In [ ]:
# site_id = "d1ee1d22-4b55-4c97-97c4-9d757144f45b"
df_coverage_site = df_coverage_0.filter(col("site")==site_id)
print("count", df_coverage_site.count() )
df_coverage_site.show()

In [ ]:
from pyspark.sql.types import IntegerType, FloatType, StringType

def mac_format(mac):
    return mac.replace("-", "")
mac_format = udf(mac_format, StringType())

df_coverage_site= df_coverage_site.withColumn("ap", mac_format(col("ap")))
df_coverage_site.show(3)                           

In [ ]:
df_scan_site = df_scan.filter(col('site')==site_id)

print("count", df_scan_site.count())

df_scan_site.show()

In [ ]:
# df_scan_site.select("ap").groupBy("ap").count().show()

df_scan_site.select(F.countDistinct("ap")).show()



In [ ]:
df_scan_site.select(F.countDistinct("ap2")).show()


In [ ]:
df_scan_site.select(F.countDistinct("ap", "ap2")).show()

# GraphFrames
The following command creates your first GraphFrame. The GraphFrame accepts two DataFrames as inputs—vertices and edges. GraphFrames like to have a naming convention in the column name, which you need to follow. Those rules are defined as follows:
* A DataFrame that represents vertices should contain a column named id. Here, personsDf contains a column name "id".

* A DataFrame that represents edges should contain columns named src and dst. Here, reationshipDf contains the columns "src" and "dst".

In [ ]:
df_scan_site.show(3)
df_coverage_site.show(2)

In [ ]:
df_scan_site.show()

In [ ]:
# 
df_scan_site_2 = df_scan_site.join(df_coverage_site, [df_scan_site.ap==df_coverage_site.ap], "inner")
df_scan_site_2.show(3)

In [ ]:
vertices= df_scan_site.selectExpr("ap as id").distinct()
vertices.show(3)

vertices_2 = vertices.join(df_coverage_site, [vertices.id==df_coverage_site.ap], "inner")
vertices_2.show(3)

In [ ]:
vertices.count(), vertices_2.count()

In [ ]:
vertices_3 = vertices.join(df_coverage_site, [vertices.id==df_coverage_site.ap], how='left')
vertices_3.show(3)
vertices_3.count()
vertices_3.select("id", "sle_coverage_anomaly_score").show(41)

In [ ]:
vertices_4 = vertices.join(df_coverage_site, [vertices.id==df_coverage_site.ap], how='right')
# vertices_4.show(3)
vertices_4.count()
vertices_4.select("id", "sle_coverage_anomaly_score").show(41)

In [ ]:
# vertices_3.select("id", "sle_coverage_anomaly_score").show(41)
# vertices_3= vertices_3.selectExpr("ap as id")

In [ ]:
# vertices_3.show()

In [ ]:
# vertices= df_scan_site.selectExpr("ap as id").distinct()

In [ ]:
# vertices.show(5)
# df_scan_site.show(5)

In [ ]:
edges=df_scan_site.filter(col("rssi")>-68)\
    .select("ap", "ap2", "rssi").groupBy("ap", "ap2").agg(F.max("rssi").alias("weight"))\
    .selectExpr("ap as src", "ap2 as dst", "weight")

edges.show(5)

In [ ]:
# enrich vertices

In [ ]:
edges.count()
# vertices_3.count(), edges.count()/vertices_3.count()

In [ ]:
# edges.show()
edges.filter(col("src")==ap203).show(50)

In [ ]:
# df_scan_site.filter(col("rssi")>-68).select("ap", "ap2", "rssi").groupBy("ap", "ap2").agg(F.max("rssi").alias("weight")).show()

In [ ]:
from graphframes import *
g = GraphFrame(vertices_3, edges)

In [ ]:
g.vertices.show()
g.edges.show()
## Check the number of edges of each vertex
g.degrees.show()

In [ ]:
display(g)

In [ ]:
aps = vertices.select("id").collect()
ap1 = aps[0][0]

In [ ]:
# The following code gets the number of incoming links to Andrew. This is obtained by calling the inDegrees method.
filter_query= "id ='{}'".format(ap1)
g.degrees.filter(filter_query).show()

In [ ]:
# The following code shows how to get the number of links coming out from Andrew using the outDegrees method .

# g.outDegrees.filter("id = '5c5b3552b5e3'").show()

ap203 = "5c5b35ae16bc"

g.degrees.filter("id ='{}'".format(ap203)).show()

In [ ]:
# g.edges.filter("id ='{}'".format(ap203))

In [ ]:
sc.setCheckpointDir('graphframes_cps')


In [ ]:
ranks = g.pageRank(resetProbability=0.10, maxIter=5)
# display(ranks.vertices.select("id","pagerank").orderBy(desc("pagerank")))


In [ ]:
display(ranks.vertices)
display(ranks.edges)

In [ ]:
vertices.show()

In [ ]:
# vertices.show()
# xap_s

In [ ]:
# Search from "Esther" for users of age < 32.

# ap1 = "5c5b3552b96c"
# ap2 = "5c5b3552b5e3"
# paths = g.bfs("id = ap1", "id = ap2")
# paths.show()

# # Specify edge filters or max path lengths.
# g.bfs("id = ap1", "id = ap2",\
#   edgeFilter="rssi > -75", maxPathLength=3)


In [ ]:
# g.connectedComponents()

stronglyConnectedComponents = g.stronglyConnectedComponents(maxIter=10)
# stronglyConnectedComponents = g.stronglyConnectedComponents(maxIter=10)
# stronglyConnectedComponents.select("id", "component").orderBy("component").show()
stronglyConnectedComponents.show()

In [ ]:
# result1.

In [ ]:

# result2.select("id", "component").orderBy("component").show()


In [ ]:
# result2

In [ ]:
# dir(result2)

In [ ]:
stronglyConnectedComponents.select("component").groupBy("component").count().orderBy("count").show()

In [ ]:
# result2.show()
stronglyConnectedComponents.count()

In [ ]:
# dir(g)

In [ ]:
connectedComponents = g.connectedComponents()
# result2.select("id", "component").orderBy("component").show()
connectedComponents.show()

In [ ]:
connectedComponents.select("ap", "component").show(connectedComponents.count())  #.groupBy("component").count().orderBy("count").show()

In [ ]:
vertices_2.select("ap").count()

In [ ]:
g.vertices.select("ap").count()

In [ ]:
date_day + hr

In [ ]:
s3_out_bucket = "s3://mist-test-bucket/wenfeng/{}/hr={}".format(date_day, hr)  #top_1_time_epoch_by_site_ap_ap2_band/{}/{}/".format(date_day, hr)
s3_out_bucket_vertices = s3_out_bucket #+ "vertices/"
s3_out_bucket_edges = s3_out_bucket #+ "edges/"

print(s3_out_bucket_vertices, s3_out_bucket_edges)
g.vertices.write.mode("overwrite").parquet(s3_out_bucket_vertices)
g.edges.write.mode("overwrite").parquet(s3_out_bucket_vertices)
# s3_out_bucket


In [ ]:
# s3_out_bucket, rrm_local_path

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline 

import networkx as nx
gp = nx.from_pandas_edgelist(edges.toPandas(),'src','dst')
nx.draw(gp, with_labels = True)

In [ ]:
# plt.figsize(15, 20)
plt.figure(figsize=(20,10)) 
nx.draw(gp, with_labels = True)

In [ ]:
ver

In [ ]:
g.persist()

In [ ]:
# graph.degrees.filter("id = 1").show()

In [ ]:
# GraphFrame provides the following built-in algorithms:
# Connected components

# Label propagation

# PageRank

# SVD++

# Shortest Path

# Strongly connected components

# Triangle count

In [ ]:
import networkx as nx
mist_g = nx.read_gpickle("../../mist-rrm-exp/test-notebooks/mistG_sams_4989.gpickle")


In [ ]:
# df_coverage_site = df_coverage_0.filter(col("site")==site_id)
df_coverage_site.show(0)